# Рекомендация тарифов


<div> <b>Цель исследования:</b><br>
    1. Проанализировать поведение клиентов<br>
    2. Предложить пользователям новый тариф<br>
    3. Построить модель с максимально большим значением <b>accuracy</b>
    <br>
    <b>Ход исследования:</b><br>
    1. Выгрузить таблицу<br>
    2. Разделить исходные данные на обучающую, валидационную и тестовую выборки<br>
    3. Построить модель для задачи классификации, которая выберет подходящий тариф<br>
    <ul>
         <li>Решающее дерево</li>
         <li>Случайный лес</li>
         <li>Логистическая регрессия</li>
    </ul>
    4. Проверить качество модели на тестовой выборке<br>
    5. Проверить модели на вменяемость<br>
</div>

## Откройте и изучите файл

Подключим все необходимые модули

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

Откроем таблицу

In [2]:
df = pd.read_csv('users_behavior.csv')
display(df.head(5))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
display(df.info())
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


данных достаточно, чтобы провести анализ

Отделим фичи:
<ul>
     <li>calls</li>
     <li>minutes</li>
     <li>messages</li>
     <li>mb_used</li>
</ul>
от таргета:
<ul>
     <li>is_ultra</li>
</ul>

In [36]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

## Разбейте данные на выборки

Сделаем 3 выборки:
<ul>
     <li>Обучающая 60%</li>
     <li>Валидационная 20%</li>
     <li>Тестовая 20%</li>
</ul>

In [38]:
features_train_full, features_test, target_train_full, target_test = train_test_split(
    features, target, test_size=0.2, stratify=target, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_full, target_train_full, test_size=0.25, stratify=target_train_full, random_state=12345)

In [39]:
print(features_train.shape, target_train.shape)
print(features_valid.shape, target_valid.shape)
print(features_test.shape, target_test.shape)

(1928, 4) (1928,)
(643, 4) (643,)
(643, 4) (643,)


размеры соответствуют требованию

Получили три выборки
<ul>
     <li>features_train, target_train</li>
     <li>features_valid, target_valid</li>
     <li>features_test, target_test</li>
</ul>

## Исследуйте модели

Построим дерево решений

In [40]:
best_model = None
best_accuracy = 0
depth_best = 0
est_best = 0
est = False

In [41]:
def model_accuracy_best(
                            model_n, 
                            features_train=features_train, 
                            target_train=target_train,
                            features_test=features_valid,
                            target_test=target_valid
                            ):
    global best_model
    global best_accuracy
    global depth_best
    global est_best
    model_n.fit(features_train,target_train)
    accuracy = accuracy_score(target_test, model_n.predict(features_test))
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model_n
        if depth:
            depth_best = depth
        if est:
            est_best = est
    return(best_accuracy, best_model, est_best, depth_best)

In [42]:
%%time

for depth in range(1,20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_accuracy_best(model)

print(best_accuracy, depth_best)
model_tree_best = best_model

0.8164852255054432 5
CPU times: total: 125 ms
Wall time: 122 ms


Лучшие результаты показывает дерево с глубиной 5.

Построим Случайный лес

In [87]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

clf = RandomForestClassifier(random_state=12345)
parametrs = { 'n_estimators': range (15, 30),
              'max_depth': range (7,12)
              #'min_samples_leaf': range (1,2),
              #'min_samples_split': range (2,3) 
            }

In [88]:
grid = GridSearchCV(clf, parametrs, cv=StratifiedKFold(n_splits=5, random_state=12345, shuffle=True))
grid.fit(features_train, target_train)

grid.best_params_

{'max_depth': 8, 'n_estimators': 19}

In [89]:
best_grid_forest = RandomForestClassifier(random_state=12345,n_estimators=19, max_depth=8)

In [91]:
accuracy_score(target_valid, grid.predict(features_valid))

0.8242612752721618

In [49]:
%%time
best_model = None
best_accuracy = 0
depth_best = 0
est_best = 0
est = True

for est in range(18, 25, 1):
    for depth in range(5,15):
        model = RandomForestClassifier(random_state=12345,n_estimators=est, max_depth=depth)
        model_accuracy_best(model)

print(best_accuracy, depth_best, est_best)
model_forest_best = best_model

0.8273716951788491 7 20
CPU times: total: 4.27 s
Wall time: 4.27 s


Лучшая модель с 20 деревьями и глубиной 7.

Построим модель логистической регрессии

In [83]:
%%time
best_model = None
best_accuracy = 0
depth_best = 0
est_best = 0
est = True

model = LogisticRegression(random_state=12345)
model_accuracy_best(model)

print(best_accuracy)
model_logreg_best = best_model

0.7558320373250389
CPU times: total: 15.6 ms
Wall time: 13.8 ms


In [98]:
grid = GridSearchCV(LogisticRegression(random_state=12345, solver='liblinear'), 
                    {'C' : [5,6, 7, 4], 'penalty': ['l1', 'l2']}, 
                    cv=StratifiedKFold(n_splits=5, random_state=12345, shuffle=True))
grid.fit(features_train, target_train)
grid.best_params_

accuracy_score(target_valid, grid.predict(features_valid))

0.7558320373250389

In [99]:
grid.best_params_

{'C': 5, 'penalty': 'l1'}

In [100]:
best_grid_lin = LogisticRegression(random_state=12345, solver='liblinear', C=5, penalty = 'l1')

Данная модель показала худшие результаты.

Лучшие результаты показал случайный лес, худшие - логистическая регрессия.

## Проверьте модель на тестовой выборке

In [101]:
model_tree_best.fit(features_train_full, target_train_full)
model_logreg_best.fit(features_train_full, target_train_full)
model_forest_best.fit(features_train_full, target_train_full)
best_grid_lin.fit(features_train_full, target_train_full)
best_grid_forest.fit(features_train_full, target_train_full)

RandomForestClassifier(max_depth=8, n_estimators=19, random_state=12345)

Проверим лучшие модели на тестовой выборке

In [102]:
print('Дерево:', accuracy_score(target_test, model_tree_best.predict(features_test)))
print('Лес:', accuracy_score(target_test, model_forest_best.predict(features_test)))
print('Регрессия:', accuracy_score(target_test, model_logreg_best.predict(features_test)))
print('Регрессия GS:', accuracy_score(target_test, best_grid_lin.predict(features_test)))
print('Лес GS:', accuracy_score(target_test, best_grid_forest.predict(features_test)))

Дерево: 0.7838258164852255
Лес: 0.8180404354587869
Регрессия: 0.7091757387247278
Регрессия GS: 0.7433903576982893
Лес GS: 0.8164852255054432


Результаты близки к результатам на валидационной выборке.

## Вывод

Дана таблица с данными о пользователях мобильной сети. 

Разбили ее на 3 группы : обучение, валидация, тест.

Были построены 3 модели на обучающей выборке: 
<ul>
     <li>Решающее дерево с глубиной 10</li>
     <li>Случайный лес с глубиной 13 и 15 деревьями</li>
     <li>Логистическая регрессия</li>
</ul>
Все модели показали неплохой результат, значительно превышающий случайный.

После этого снова обучили модели уже на объединении обучающей и валидационной выборках.

Все модели показали результат лучше.

Лучшие результаты получены для модели случайного леса. Построение данной модели заняло много времени, но считаю несколько секунд несущественными в данном случае.